In [1]:
%autosave 0

Autosave disabled


# Die Onkologie-Station

Auf einer Onkologie-Station liegen fünf Patienten in nebeneinander liegenden Zimmern.
Bis auf einen  der Patienten hat jeder genau eine Zigaretten-Marke geraucht.
Der Patient, der nicht Zigarette geraucht hat, hat Pfeife geraucht.
Jeder Patient fährt genau ein Auto und ist
an genau einer Krebs-Art erkrankt.  Zusätzlich haben Sie die folgenden Informationen:
<ol>
<li> Im Zimmer neben Michael wird Camel geraucht. </li>
<li> Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
      Zungen-Krebs Patienten. </li>
<li> Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs. </li>
<li> Der West-Raucher liegt im ersten Zimmer. </li>
<li> Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. </li>
<li> Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. </li>
<li> Rudolf wünscht sich Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. </li>
<li> Der Seat Fahrer hat morgen seinen letzten Geburtstag. </li>
<li> Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. </li>
<li> Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. </li>
<li> Der Nissan Fahrer liegt neben dem Mazda-Fahrer. </li>
<li> Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. </li>
<li> Jens liegt neben dem Luckies Raucher. </li>
<li> Der Hodenkrebs-Patient hat gestern seine Eier durchs Klo gespült. </li>
</ol>
Entwickeln Sie ein <em>Python</em>-Programm, das die folgenden Fragen beantwortet:
<ol> 
<li> Was raucht der Darmkrebs-Patient? </li>
<li> Was fährt Kurt für ein Auto? </li>
</ol>

## Importing the Necessary Modules

Our goal is to solve this puzzle by first coding it as a solvability problem of propositional logic and then to solve the resulting set of clauses using the algorithm of Davis and Putnam.

In [2]:
import davisPutnam as dp

In order to be able to transform formulas from propositional logic into sets of clauses we import the module <tt>cnf</tt> which implements the function <tt>normalize</tt> that takes a formula and transforms it into a set of clauses.

In [3]:
import cnf

In order to write formulas conveniently, we use the parser for propositional logic.

In [4]:
import propLogParser as plp

Using the parser and the module <tt>cnf</tt> we can impement a function $\texttt{parseCNF}(s)$ that takes a string $s$ representing a formula and transforms $s$ into an equivalent set of clauses.

In [5]:
def parseCNF(s):
    nestedTuple = plp.LogicParser(s).parse()
    Clauses     = cnf.normalize(nestedTuple)
    return Clauses

## Auxiliary Functions

In [6]:
def atMostOne(V):
    return { frozenset({('¬',p), ('¬', q)}) for p in V
                                            for q in V 
                                            if  p != q 
           }

Given a name $f$ and an index $i \in\{1,2,3,4,5\}$, the function $\texttt{var}(i)$ creates the string 
$f\langle i \rangle$, e.g. the call <tt>var("Japanese", 2)</tt> returns the following string:

<tt>Japanese$\langle$2$\rangle$</tt>.

In [7]:
def var(f, i):
    return f + "<" + str(i) + ">" 

In [8]:
var("Japanese", 2)

'Japanese<2>'

The call $\texttt{flatten}(\texttt{LoS})$ takes list of sets $\texttt{LoS}$ and adds all the sets in this list into one big set. 

In [9]:
def flatten(ListOfSets):
    return {x for S in ListOfSets for x in S}

In [10]:
flatten([{1,2,3}, {3,4,5}])

{1, 2, 3, 4, 5}

A call of the form $\texttt{x}$ will return a clause that specifies that the person with property $x$ has to live in one of the houses from $1$ to $5$.

In [11]:
def somewhere(x):
    return frozenset({ var(x, i) for i in range(1, 5+1) })

In [12]:
somewhere("a")

frozenset({'a<1>', 'a<2>', 'a<3>', 'a<4>', 'a<5>'})

Given an exclusive set of properties $S$ and a house number $i$, the function $\texttt{atMostOne}(S, i)$ returns a set of clauses that specifies that the person living in house number $i$ has at most one of the properties from the set $S$.  For example, if 
$S = \{\texttt{"Japanese"}, \texttt{"Englishman"}, \texttt{"Spaniard"}, \texttt{"Norwegian"}, \texttt{"Ukranian"}\}$, 
then $\texttt{atMostOne}(S, 3)$ specifies that the inhabitant of house number 3 has at most one of the nationalities from the set $S$.

In [13]:
def atMostOneAt(S, i):
    return atMostOne({ var(x, i) for x in S })

In [14]:
atMostOneAt({"A", "B", "C"}, 1)

{frozenset({('¬', 'B<1>'), ('¬', 'C<1>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'C<1>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'B<1>')})}

Implement a function $\texttt{onePerHouse}(S)$ which could be called as follows:
$$\texttt{onePerHouse}(\{\texttt{"Japanese"},
       \texttt{"Englishman"}, 
       \texttt{"Spaniard"}, \texttt{"Norwegian"}, 
       \texttt{"Ukranian"}\})
$$
This function would create a set of clauses that expresses that there has to be a house where the Japanese lives, a house where the Englishman lives, a house where the Spaniard lives, a house where the Norwegian lives, and a house
where the Ukranian lives.  Furthermore, the set of clauses would create clauses that express that these five persons live in **different** houses.

When implementing this function, you should use the functions <tt>somewhere</tt> and <tt>atMostOne</tt>.

In [15]:
def onePerRoom(S):
    return { somewhere(x) for x in S } | flatten([ atMostOneAt(S, i) for i in range(1, 5+1) ])

In [16]:
onePerRoom({"A", "B", "C", "D", "E"})

{frozenset({('¬', 'C<2>'), ('¬', 'E<2>')}),
 frozenset({('¬', 'B<4>'), ('¬', 'C<4>')}),
 frozenset({('¬', 'A<5>'), ('¬', 'E<5>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'B<1>')}),
 frozenset({('¬', 'C<5>'), ('¬', 'E<5>')}),
 frozenset({('¬', 'A<3>'), ('¬', 'D<3>')}),
 frozenset({('¬', 'A<4>'), ('¬', 'E<4>')}),
 frozenset({('¬', 'A<4>'), ('¬', 'C<4>')}),
 frozenset({('¬', 'B<1>'), ('¬', 'D<1>')}),
 frozenset({('¬', 'A<2>'), ('¬', 'D<2>')}),
 frozenset({('¬', 'B<2>'), ('¬', 'E<2>')}),
 frozenset({('¬', 'D<1>'), ('¬', 'E<1>')}),
 frozenset({('¬', 'A<5>'), ('¬', 'B<5>')}),
 frozenset({('¬', 'A<3>'), ('¬', 'B<3>')}),
 frozenset({'B<1>', 'B<2>', 'B<3>', 'B<4>', 'B<5>'}),
 frozenset({('¬', 'D<5>'), ('¬', 'E<5>')}),
 frozenset({'D<1>', 'D<2>', 'D<3>', 'D<4>', 'D<5>'}),
 frozenset({('¬', 'B<5>'), ('¬', 'E<5>')}),
 frozenset({('¬', 'B<3>'), ('¬', 'C<3>')}),
 frozenset({('¬', 'B<1>'), ('¬', 'E<1>')}),
 frozenset({('¬', 'C<1>'), ('¬', 'D<1>')}),
 frozenset({('¬', 'B<5>'), ('¬', 'C<5>')}),
 frozenset({

Given to properties $a$ and $b$ the function $\texttt{sameHouse}(a, b)$ computes a set of clauses that specifies that if the inhabitant of house number $i$ has the property $a$, then he also has the property $b$ and vice versa.  For example, $\texttt{sameHouse}(\texttt{"Japanese"}, \texttt{"Dog"})$ specifies that the Japanese guy keeps a dog.

In [17]:
def sameRoom(a, b):
    return flatten([ parseCNF(f"{var(a, i)} ↔ {var(b, i)}") for i in range(1, 5+1) ])

In [18]:
sameRoom("Luckies", "Camel")

{frozenset({'Camel<5>', ('¬', 'Luckies<5>')}),
 frozenset({'Camel<4>', ('¬', 'Luckies<4>')}),
 frozenset({'Camel<1>', ('¬', 'Luckies<1>')}),
 frozenset({('¬', 'Luckies<2>'), 'Camel<2>'}),
 frozenset({'Camel<3>', ('¬', 'Luckies<3>')}),
 frozenset({'Luckies<3>', ('¬', 'Camel<3>')}),
 frozenset({'Luckies<4>', ('¬', 'Camel<4>')}),
 frozenset({'Luckies<1>', ('¬', 'Camel<1>')}),
 frozenset({('¬', 'Camel<5>'), 'Luckies<5>'}),
 frozenset({('¬', 'Camel<2>'), 'Luckies<2>'})}

Given to properties $a$ and $b$ the function $\texttt{nextTo}(a, b)$ computes a set of clauses that specifies that the inhabitants with properties $a$ and $b$ are direct neighbours.  For example, $\texttt{nextTo}(\texttt{'Japanese'}, \texttt{'Dog'})$ specifies that the Japanese guy lives next to the guy who keeps a dog.

In [19]:
def differentRoom(a, b):
    return { frozenset({ ('¬', var(a, i)), ('¬', var(b, i)) }) for i in range(1, 5+1) }

In [20]:
differentRoom('A', 'B')

{frozenset({('¬', 'A<4>'), ('¬', 'B<4>')}),
 frozenset({('¬', 'A<1>'), ('¬', 'B<1>')}),
 frozenset({('¬', 'A<5>'), ('¬', 'B<5>')}),
 frozenset({('¬', 'A<2>'), ('¬', 'B<2>')}),
 frozenset({('¬', 'A<3>'), ('¬', 'B<3>')})}

In [21]:
def nextTo(a, b):
    Result = parseCNF(f"{var(a,1)} → {var(b,2)}")
    for i in [2, 3, 4]:
        Result |= parseCNF(f"{var(a,i)} → {var(b,i-1)} ∨ {var(b,i+1)}")
    Result |= parseCNF(f"{var(a,5)} → {var(b,4)}")
    return Result

In [22]:
nextTo('A', 'B')

{frozenset({'B<1>', ('¬', 'A<2>'), 'B<3>'}),
 frozenset({('¬', 'A<5>'), 'B<4>'}),
 frozenset({'B<5>', ('¬', 'A<4>'), 'B<3>'}),
 frozenset({'B<2>', ('¬', 'A<3>'), 'B<4>'}),
 frozenset({'B<2>', ('¬', 'A<1>')})}

In [23]:
def allClauses():
    Brands   = { "Camel", "Ernte", "West", "Luckies", "Pfeife" }
    Cars     = { "Trabant", "Mazda", "Nissan", "Seat", "Mercedes" }
    Cancers  = { "Zunge", "Kehlkopf", "Lunge", "Darm", "Hoden" }
    Names    = { "Michael", "Rolf", "Rudolf", "Jens", "Kurt" }
    Clauses  = onePerRoom(Brands)
    Clauses |= onePerRoom(Cars)
    Clauses |= onePerRoom(Cancers)
    Clauses |= onePerRoom(Names)
    # Im Zimmer neben Michael wird Camel geraucht. 
    Clauses |= nextTo("Michael", "Camel")
    # Der Trabant-Fahrer raucht Ernte 23 und liegt im Zimmer neben dem 
    # Zungen-Krebs Patienten. 
    Clauses |= sameRoom("Trabant", "Ernte")
    Clauses |= nextTo("Trabant", "Zunge")
    # Rolf liegt im letzten Zimmer und hat Kehlkopf-Krebs. 
    Clauses |= { frozenset({ "Rolf<5>" }) }
    Clauses |= sameRoom("Rolf", "Kehlkopf")
    # Der West-Raucher liegt im ersten Zimmer. 
    Clauses |= { frozenset({ "West<1>" }) }
    # Der Mazda-Fahrer hat Zungen-Krebs und liegt neben dem Trabant-Fahrer. 
    Clauses |= sameRoom("Mazda", "Zunge")
    Clauses |= nextTo("Mazda", "Trabant")
    # Der Nissan-Fahrer liegt neben dem Zungen-Krebs Patient. 
    Clauses |= nextTo("Nissan", "Zunge")
    # Rudolf wünscht sich Sterbe-Hilfe und liegt zwischen dem Camel-Raucher und dem Trabant-Fahrer. 
    Clauses |= nextTo("Rudolf", "Camel")
    Clauses |= nextTo("Rudolf", "Trabant")
    Clauses |= differentRoom("Camel", "Trabant")
    # Der Luckies Raucher liegt neben dem Patienten mit Lungen-Krebs. 
    Clauses |= nextTo("Luckies", "Lunge")
    # Der Camel Raucher liegt neben dem Patienten mit Darm-Krebs. 
    Clauses |= nextTo("Camel", "Darm")
    # Der Nissan Fahrer liegt neben dem Mazda-Fahrer. 
    Clauses |= nextTo("Nissan", "Mazda")
    # Der Mercedes-Fahrer raucht Pfeife und liegt neben dem Camel Raucher. 
    Clauses |= sameRoom("Mercedes", "Pfeife")
    Clauses |= nextTo("Mercedes", "Camel")
    # Jens liegt neben dem Luckies Raucher. 
    Clauses |= nextTo("Jens", "Luckies")
    return Clauses

In [24]:
Clauses = allClauses()
Clauses

{frozenset({'Zunge<3>', 'Zunge<1>', ('¬', 'Nissan<2>')}),
 frozenset({('¬', 'Hoden<5>'), ('¬', 'Zunge<5>')}),
 frozenset({('¬', 'Hoden<5>'), ('¬', 'Lunge<5>')}),
 frozenset({('¬', 'Seat<1>'), ('¬', 'Trabant<1>')}),
 frozenset({('¬', 'Michael<3>'), 'Camel<4>', 'Camel<2>'}),
 frozenset({'Ernte<4>', ('¬', 'Trabant<4>')}),
 frozenset({('¬', 'Nissan<5>'), 'Mazda<4>'}),
 frozenset({'Mercedes<1>', ('¬', 'Pfeife<1>')}),
 frozenset({('¬', 'Mazda<4>'), ('¬', 'Nissan<4>')}),
 frozenset({('¬', 'Camel<1>'), ('¬', 'Luckies<1>')}),
 frozenset({('¬', 'Camel<4>'), ('¬', 'Pfeife<4>')}),
 frozenset({'Kehlkopf<4>', ('¬', 'Rolf<4>')}),
 frozenset({('¬', 'Hoden<3>'), ('¬', 'Kehlkopf<3>')}),
 frozenset({('¬', 'Lunge<5>'), ('¬', 'Zunge<5>')}),
 frozenset({('¬', 'Luckies<1>'), ('¬', 'Pfeife<1>')}),
 frozenset({('¬', 'Ernte<5>'), 'Trabant<5>'}),
 frozenset({('¬', 'Kehlkopf<5>'), ('¬', 'Zunge<5>')}),
 frozenset({('¬', 'Camel<3>'), ('¬', 'Luckies<3>')}),
 frozenset({('¬', 'Nissan<5>'), 'Zunge<4>'}),
 frozenset({'

In [25]:
len(Clauses)

322

In [26]:
def solve():
    Clauses = allClauses()
    return dp.solve(Clauses, set())

In [27]:
import time

Solving the problem takes about 7 seconds on my computer.

In [28]:
start    = time.time()
Solution = solve()
stop     = time.time()
print(f'Time needed: {round((stop-start)*10)/10} seconds.')
Solution

Time needed: 0.4 seconds.


{frozenset({('¬', 'Rudolf<3>')}),
 frozenset({('¬', 'Kehlkopf<3>')}),
 frozenset({'Nissan<3>'}),
 frozenset({('¬', 'Ernte<2>')}),
 frozenset({('¬', 'Darm<4>')}),
 frozenset({'Rudolf<4>'}),
 frozenset({'Seat<1>'}),
 frozenset({('¬', 'Zunge<2>')}),
 frozenset({('¬', 'Hoden<4>')}),
 frozenset({('¬', 'Hoden<5>')}),
 frozenset({('¬', 'Rudolf<1>')}),
 frozenset({'West<1>'}),
 frozenset({('¬', 'Rolf<3>')}),
 frozenset({('¬', 'Kurt<3>')}),
 frozenset({'Lunge<3>'}),
 frozenset({('¬', 'Jens<1>')}),
 frozenset({('¬', 'Jens<4>')}),
 frozenset({('¬', 'Darm<5>')}),
 frozenset({'Camel<3>'}),
 frozenset({'Jens<3>'}),
 frozenset({('¬', 'Rolf<4>')}),
 frozenset({('¬', 'Kurt<4>')}),
 frozenset({('¬', 'Seat<5>')}),
 frozenset({('¬', 'Trabant<4>')}),
 frozenset({'Darm<2>'}),
 frozenset({('¬', 'Rudolf<2>')}),
 frozenset({('¬', 'Nissan<4>')}),
 frozenset({('¬', 'Camel<1>')}),
 frozenset({'Mazda<4>'}),
 frozenset({('¬', 'Zunge<5>')}),
 frozenset({('¬', 'Luckies<1>')}),
 frozenset({('¬', 'Mazda<1>')}),
 frozen

## Functions to PrettyPrint the Solution

In [29]:
def arb(S):
    for x in S:
        return x

In [30]:
def pad(s, l):
    n = l - len(s)
    return s + " " * n

In [31]:
def join(L, sep):
    result = ''
    for s in L:
        result += s + sep
    if len(L) > 0:
        result = result[:-len(sep)]
    return result

In [32]:
def printSolution(UnitClauses):
    Brands     = { "Camel", "Ernte", "West", "Luckies", "Pfeife"    }
    Cars       = { "Trabant", "Mazda", "Nissan", "Seat", "Mercedes" }
    Cancers    = { "Zunge", "Kehlkopf", "Lunge", "Darm", "Hoden" }
    Names      = { "Michael", "Rolf", "Rudolf", "Jens", "Kurt" }
    Assignment = {}
    CarsDict   = {}
    BrandsDict = {}
    for Unit in UnitClauses:
        Literal = arb(Unit)
        if isinstance(Literal, str):
            number = int(Literal[-2])
            name   = Literal[:-3]
            Assignment[name] = number
            if name in Brands:
                BrandsDict[number] = name
            if name in Cars:
                CarsDict[number] = name
    print()

    print(f"Der Darmkrebs-Patient raucht {BrandsDict[Assignment['Darm']]}.");
    print(f"Kurt fährt einen {CarsDict[Assignment['Kurt']]}.");
    print()
    longest = max({ len(x) for x in Assignment })
    line    = "-" * (4 * (longest + 3) + 6);
    for house in range(1, 5+1):
        print(line)
        l = [];
        for Class in [Brands, Cars, Cancers, Names]:
            for x in Class:
                l += [ pad(f"{y}", longest) for y in Assignment 
                                            if y == x and Assignment[y] == house
                     ]
        print(f"| {house}: | " + join(l, " | ") + " |")
    print(line)
    print()

In [33]:
printSolution(Solution)


Der Darmkrebs-Patient raucht Pfeife.
Kurt fährt einen Seat.

--------------------------------------------------
| 1: | West     | Seat     | Hoden    | Kurt     |
--------------------------------------------------
| 2: | Pfeife   | Mercedes | Darm     | Michael  |
--------------------------------------------------
| 3: | Camel    | Nissan   | Lunge    | Jens     |
--------------------------------------------------
| 4: | Luckies  | Mazda    | Zunge    | Rudolf   |
--------------------------------------------------
| 5: | Ernte    | Trabant  | Kehlkopf | Rolf     |
--------------------------------------------------



## Checking the Uniqueness of the Solution

Given a set of unit clauses $U$, the function $\texttt{checkUniqueness}(U)$ returns a clause that is the negation of the set $U$.

In [34]:
def negateSolution(UnitClauses):
    return { dp.complement(arb(unit)) for unit in UnitClauses }

In [35]:
negateSolution({ frozenset({'a'}), frozenset({('¬', 'b')}) }) 

{('¬', 'a'), 'b'}

The function $\texttt{checkUniqueness}(\texttt{Solution}, \texttt{Clauses})$  takes a set of $\texttt{Clauses}$ and a $\texttt{Solution}$ for these clauses and checks, whether this is the only solution.

In [36]:
def checkUniqueness(Solution, Clauses):
    negation = negateSolution(Solution)
    Clauses.add(frozenset(negation))
    alternative = dp.solve(Clauses, set())
    if alternative == { frozenset() }:
        print("Well done: The solution is unique!")
    else:
        print("ERROR: The solution is not unique!")

In [37]:
checkUniqueness(Solution, Clauses)

Well done: The solution is unique!
